# FROM VSCODE TO RU_VIZ DIR

In [2]:
# disable warnings up front for a cleaner experience
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Interactive Visualization

We have seen basic examples of visualizing data, and we have seen that using the `bokeh` plotting API (under `holoviews`) allows us to interact with our visualizations on a basic level (pan, zoom, etc.). This is only the surface though! We can create *widgets* to give the viewer controls and tools for manipulating the data and visualization. This is the true power of data visualization in Python using `holoviews` and a new library called `panel`.

## NOTE

Support for interactive widgets in VSCode/Codespaces is a little low compared directly to Jupyter. Things will work, but will occassionally break and require restarting the kernel. Using Jupyter explictly is strongly preferred here. To help mitigate issues in VS Code/Codespaces we are going to repeat a lot of code (the code we will repeat keeps the internal complexities in check to an extent). 

### Imports & Data

First thing we need to so is import a few things to get started:

In [3]:
import holoviews as hv
import hvplot.pandas
import numpy as np
import pandas as pd
import panel as pn

In [4]:
%%capture
# disables output
# the above is a magic function. more on magic functions here
# https://ipython.readthedocs.io/en/stable/interactive/magics.html#cell-magics
hv.extension('bokeh')
pn.extension(comms="vscode")

This will prep panel to display the graphs for the HTML, CSS etc 

We need to work with some data - this arbitary dataset randomly generates labels from a categorical list, assigns a sequential ID, and randomly sets a value uniformly distributed between 0.0 and 1.0.

In [5]:
data = pd.read_csv('data/basic.csv')
data

,label,id,x,y
0,bar,0,0.688027,0.929612
1,baz,1,0.196387,0.810796
2,foo,2,0.894665,0.774059
3,foo,3,0.482647,0.181535
4,baz,4,0.055227,0.625524
...,...,...,...,...
995,foo,995,0.049635,0.139168
996,foo,996,0.969112,0.213887
997,bar,997,0.394580,0.002596
998,bar,998,0.885870,0.313760


## Widgets

First let's plot our data using `hvplot`

In [6]:
data.hvplot.scatter(
    x='x',
    y='y',
    color='label', # Hv plot is smart enough to tell that these are not colors but rather they are different values 
).opts(
    title='Random Data',
    show_grid=True,
)

:Scatter   [x]   (y,label)


It is simple enough to apply filters to the data and replot:

In [7]:
mask = (data.label == 'bar') & (data.y > 0.25) & (data.y < 0.75)
data[mask].hvplot.scatter(
    x='x',
    y='y',
    color='label'
).opts(
    title='Random Data: 0.25 < bar < 0.75',
    show_grid=True
)

:Scatter   [x]   (y,label)

It immediately becomes a little cumbersome to remask and replot our data whenever we wish to view a different aspect of it. What if we have a way to select what labels we are viewing, and what range of the x and y values to consider? `panel` let's us do exactly that!

We need two kinds of widgets here:

* `FloatSlider`
* `Select`

In [8]:
label_selector = pn.widgets.Select(   # create a Select widget
    name='Label',                     # name of the widget
    options=['foo', 'bar', 'baz'],    # options for users to select from
)
x_slider = pn.widgets.FloatSlider( # create a FloatSlider
    name='Min x',                  # name of the widget
    start=0.0,                     # lower bound of the slider
    end=1.0,                       # upper bound of the slider
    step=0.01,                     # step value
    value=0.0                      # default value of the slider
)
y_slider = pn.widgets.FloatSlider( # create a FloatSlider
    name='Min Y',                  # name of the widget
    start=0.0,                     # lower bound of the slider
    end=1.0,                       # upper bound of the slider
    step=0.01,                     # step value
    value=0.0                      # default value of the slider
)
# t_slider = pn.widgets.FloatSlider( # create a FloatSlider
#     name='Min Y',                  # name of the widget
#     start=0.0,                     # lower bound of the slider
#     end=1.0,                       # upper bound of the slider
#     step=0.01,                     # step value
#     value=0.0                      # default value of the slider
# )

pn.Row(
    label_selector,
    # t_slider,
    pn.Column(    
    x_slider,
    y_slider)
)


BokehModel(combine_events=True, render_bundle={'docs_json': {'637b18f8-d768-4312-a2f8-d53678bef436': {'version…

```import folium
import panel as pn

pn.extension(sizing_mode="stretch_width")
m = folium.Map(location=[52.51, 13.39], zoom_start=12)

folium_pane = pn.pane.plot.Folium(m, height=400)

folium_pane
# Add a marker to the map
folium.Marker(
    [52.516, 13.381], popup="<i>Brandenburg Gate</i>", tooltip="Click me!"
).add_to(m)

folium_pane.object = m```

Cool right? Well it is, but we have not done anything yet! We need to tie these widgets to some visualization. `panel` makes this easy too. We just need to create a function to create the plot for us using values from our widgets!

In [9]:
def plot_my_data(label, min_x, min_y):
    
    # uses `data` from the global scope!
    mask = (data.label == label) & (data.x >= min_x) & (data.y >= min_y)
    return data[mask].hvplot.scatter(
        x='x',
        y='y',
        color='label'
    ).opts(
        title='Random Data',
        show_grid=True,
        framewise=True # This is extra 
    )

label_selector = pn.widgets.Select(   # create a Select widget
    name='Label',                     # name of the widget
    options=['foo', 'bar', 'baz'],    # options for users to select from
)
x_slider = pn.widgets.FloatSlider( # create a FloatSlider
    name='Min X',                  # name of the widget
    start=0.0,                     # lower bound of the slider
    end=1.0,                       # upper bound of the slider
    step=0.01,                     # step value
    value=0.0                      # default value of the slider
)
y_slider = pn.widgets.FloatSlider( # create a FloatSlider
    name='Min Y',                  # name of the widget
    start=0.0,                     # lower bound of the slider
    end=1.0,                       # upper bound of the slider
    step=0.01,                     # step value
    value=0.0                      # default value of the slider
)

################################################################################################
# This is where we stitch together the panel and the scatter graphs 

# we use `pn.interact` to connect our widgets to our plotting function
# every argument in our function gets mapped to a widget using keyword-arguments
interaction = pn.interact(plot_my_data, label=label_selector, min_x=x_slider, min_y=y_slider)
interaction

BokehModel(combine_events=True, render_bundle={'docs_json': {'faa29425-c5d6-423c-9d2d-c26655ea7d15': {'version…

Woo! Interactive visualization! What are some issues here though?

<details>
<summary>SPOILER ALERT!</summary>

Visually there are some issues...

* No matter which label we pick, the scatter color is always blue!
* As we slide values for x and y, the axis limits change!
* If we pan or zoom, and then interact with the plot, the pan and zoom are reset!

Functionally there are some issues...

* We cannot select more than one label!
* We are only controlling the minimum values for x, it would be nice to be able to control the maximum values too!

</details>

## Exercise

Let's address the issues noted above.

1. For the colors we need to define and supply a `cmap`; for this it is just a dictionary of labels to colors. This is provided as `cmaps` to the `opts` function
2. For the axis limits we need to anchor them to the bounds of our data. This is provided as `xlim` and `ylim` to the `opts` function

In [10]:
def plot_my_data(label, min_x, min_y):
    cmap = {'foo':'red','bar':'green', 'baz':'blue'}
    # uses `data` from the global scope!
    mask = (data.label == label) & (data.x >= min_x) & (data.y >= min_y)
    return data[mask].hvplot.scatter(
        x='x',
        y='y',
        color='label'
    ).opts(
        title='Random Data',
        show_grid=True,
        framewise=True, # This is extra 
        cmap = cmap,
        xlim = (0.0, 1.0),
        ylim = (0.0,1.0),

    )

label_selector = pn.widgets.Select(   # create a Select widget
    name='Label',                     # name of the widget
    options=['foo', 'bar', 'baz'],    # options for users to select from
)
x_slider = pn.widgets.FloatSlider( # create a FloatSlider
    name='Min X',                  # name of the widget
    start=0.0,                     # lower bound of the slider
    end=1.0,                       # upper bound of the slider
    step=0.01,                     # step value
    value=0.0                      # default value of the slider
)
y_slider = pn.widgets.FloatSlider( # create a FloatSlider
    name='Min Y',                  # name of the widget
    start=0.0,                     # lower bound of the slider
    end=1.0,                       # upper bound of the slider
    step=0.01,                     # step value
    value=0.0                      # default value of the slider
)

################################################################################################
# This is where we stitch together the panel and the scatter graphs 

# we use `pn.interact` to connect our widgets to our plotting function
# every argument in our function gets mapped to a widget using keyword-arguments
interaction = pn.interact(plot_my_data, label=label_selector, min_x=x_slider, min_y=y_slider)
interaction

BokehModel(combine_events=True, render_bundle={'docs_json': {'8c3ff4b8-cdc7-46f2-a910-c45d6f8c69e6': {'version…

## Exercise

Let's use some diffrent widgets to handle the two functional issues. Copy the initial interactive plotting code, but let's create some new widgets to handle the features we are looking for.

1. Use a `MultiSelect` widget. Note that the value of a `MultiSelect` widget is a tuple of values, so when creating the mask we should use the `isin` operator on the `label` column
2. Use a `RangeSlider` widget. Note that the value of a `RangeSlider` widget is a tuple of values (min/max), so when creating the mask we need to index into the tuple to get the bounds

```
a = [1,2,3,4]

foo (a) = foo([1,2,3,4])

foo(*a) = foo(1,2,3,4)
```


In [11]:
def plot_my_data(label, x_range, y_range):
    cmap = {'foo':'red','bar':'green', 'baz':'blue'}
    # uses `data` from the global scope!
    ################################################# * this means we are expanding the list 
    mask = (data.label.isin(label)) & data.x.between(*x_range) & data.y.between(*y_range)
    return data[mask].hvplot.scatter(
        x='x',
        y='y',
        color='label'
    ).opts(
        title='Random Data',
        show_grid=True,
        framewise=True, # This is extra 
        cmap = cmap,
        xlim = (0.0, 1.0),
        ylim = (0.0,1.0),

    )

label_selector = pn.widgets.MultiSelect(   # create a Select widget
    name='Label',                     # name of the widget
    options=['foo', 'bar', 'baz'],    # options for users to select from
)
x_slider = pn.widgets.RangeSlider( # create a FloatSlider
    name='Min X',                  # name of the widget
    start=0.0,                     # lower bound of the slider
    end=1.0,                       # upper bound of the slider
    step=0.01,                     # step value
    value= (0.0,1.0)                    # default value of the slider
)
y_slider = pn.widgets.RangeSlider( # create a FloatSlider
    name='Min Y',                  # name of the widget
    start=0.0,                     # lower bound of the slider
    end=1.0,                       # upper bound of the slider
    step=0.01,                     # step value
    value= (0.0,1.0)                     # default value of the slider
)

################################################################################################
# This is where we stitch together the panel and the scatter graphs 

# we use `pn.interact` to connect our widgets to our plotting function
# every argument in our function gets mapped to a widget using keyword-arguments
interaction = pn.interact(plot_my_data, label=label_selector, x_range=x_slider, y_range=y_slider)
interaction

BokehModel(combine_events=True, render_bundle={'docs_json': {'233450a8-4913-4289-b639-0cc936845bcc': {'version…

It may be desirable to rearrange our widgets and plots, since the default layout may not be that great. Let's rearrange this so that our widgets are to the right of the plot:

In [12]:
def plot_my_data(labels, x_range, y_range):
    # uses `data` from the global scope!
    mask = (data.label.isin(labels)) & data.x.between(*x_range) & data.y.between(*y_range)
    return data[mask].hvplot.scatter(
        x='x',
        y='y',
        color='label'
    ).opts(
        cmap={'foo': 'red', 'bar': 'green', 'baz': 'blue'},
        xlim=(0,1),
        ylim=(0,1),
        title='Random Data',
        show_grid=True,
        framewise=True
    )

label_selector = pn.widgets.MultiSelect(   # create a MultiSelect widget
    name='Label',                     # name of the widget
    options=['foo', 'bar', 'baz'],    # options for users to select from
)
x_slider = pn.widgets.RangeSlider( # create a RangeSlider
    name='X-Range',                # name of the widget
    start=0.0,                     # lower bound of the slider
    end=1.0,                       # upper bound of the slider
    step=0.01,                     # step value
    value=(0.0, 1.0)               # default value of the slider
)
y_slider = pn.widgets.RangeSlider( # create a RangeSlider
    name='Y-Range',                # name of the widget
    start=0.0,                     # lower bound of the slider
    end=1.0,                       # upper bound of the slider
    step=0.01,                     # step value
    value=(0.0, 1.0)               # default value of the slider
)
print(label_selector.values)

interaction = pn.interact(plot_my_data, labels=label_selector, x_range=x_slider, y_range=y_slider)

# let's print the interaction and observe its contents
print(interaction)

['foo', 'bar', 'baz']
Column
    [0] Column
        [0] MultiSelect(name='Label', options=['foo', 'bar', 'baz'])
        [1] RangeSlider(name='X-Range', step=0.01)
        [2] RangeSlider(name='Y-Range', step=0.01)
    [1] Row
        [0] HoloViews(Scatter, name='interactive01641')


We can see that the interaction is just a *column* consisting of another sub-column (containing our widgets) and a sub-row (containing our plot). We can index the interaction column like a list, and recompose them!

In [13]:
pn.Row(
    interaction[1],
    interaction[0]
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'9bc4eeed-c052-4f44-86fa-65510c87ee6d': {'version…

## Dynamic Plots

There is a more concise way to create our interactive plots that provides more flexibilty, especially when it comes time for composing complex interactive graphics. `holoviews` provides us with a *dynamic map* type that, along with `panel`, can be set up to connect many widgets and graphics together. The code largely looks the same, but there are some notable differences. 

In [14]:
# define the widgets first!
label_selector = pn.widgets.MultiSelect(
    name='Label',
    options=['foo', 'bar', 'baz'],
)
x_slider = pn.widgets.RangeSlider(
    name='X-Range',
    start=0.0,
    end=1.0,
    step=0.01,
    value=(0.0, 1.0)
)
y_slider = pn.widgets.RangeSlider(
    name='Y-Range',
    start=0.0,
    end=1.0,
    step=0.01,
    value=(0.0, 1.0)
)


# tell the plotting up front what it depends on!
# @ symbols are the decorators for the graphs. 
# Rather than the interact methods, we use the  @ Decorators 
@pn.depends(labels=label_selector, x_range=x_slider, y_range=y_slider)
def plot_my_data(labels, x_range, y_range):
    # uses `data` from the global scope!
    mask = (data.label.isin(labels)) & data.x.between(*x_range) & data.y.between(*y_range)
    return data[mask].hvplot.scatter(
        x='x',
        y='y',
        color='label'
    ).opts(
        cmap={'foo': 'red', 'bar': 'green', 'baz': 'blue'},
        title='Random Data',
    )

# contruct a `hv.DynamicMap` from out plotting function!
pn.Row(
    hv.DynamicMap(plot_my_data),
    pn.Column(
        label_selector,
        x_slider,
        y_slider
    )
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'e8b04164-0b44-4e70-b007-51058f99815c': {'version…

Did you notice anything else different about this?

<details>
<summary>SPOILER ALERT!</summary>

Our plot updates *much smoother*.

</details>

 But why?

<details>
<summary>SPOILER ALERT!</summary>

The `hv.DynamicMap` is a special object that is optimized in a specific way that not only listens to widget updates, but it also tells holoviews *to plot the data only once*, and subsequent updates simply *redraws the data*. Plotting is a very expensive operation, and updating the underlying data is more efficient than redrawing the entire plot!

</details>

## The interact replot the graphs everytime we make a change panal.interact gives a new graphs 
## The @decorator, only changes points 

This is deeply important to real-time interactive graphics. We want the user experience to be as smooth as possible, and `hv.DynamicMap` gives us that.

From here the possibilities are seemingly endless. Any plot we can create in `holoviews` (https://holoviews.org/reference/index.html) we can interact with using `panel` (https://panel.holoviz.org/reference/index.html#). The only thing we need to aware of is how much computation we are putting in our plotting calls. We want to minimize any and all computations to avoid lag and slowdowns during visualization.

There are tons of graphics, widgets, and layouts that we can use to tremendous effect to compose highly interactive graphics. This only barely scratches the surface!

https://colorcet.holoviz.org/